In [1]:
import bs4 as bs
import pandas as pd
import matplotlib as plt
from matplotlib import pyplot as pplt
import os
from pptx import Presentation
import pprint as p

#Die Buch-Kürzel des alten Testaments
AT = ["Gn", "Ex", "Lv", "Nm", "Dt", "Ios", "Idc", "Rt", "1Sm", "2Sm", "3Rg", "4Rg", "1Par", "2Par", "Esr", "Neh", "Tb",
      "Idt", "Est", "1Mcc"
    , "2Mcc", "3Mcc", "4Mcc", "Iob", "Ps", "Prv", "Ecl", "Ct", "Sap", "Sir", "Is", "Ier", "Lam", "Bar", "Ez", "Dn",
      "Os", "Ioel", "Am", "Abd"
    , "Ion", "Mi", "Na", "Hab", "So", "Agg", "Za", "Mal"]

In [2]:
def percent_calc(num1, num2):
    '''returns percentage value'''
    return (num1/num2)*100
    
 
    
def analyse(filename):
    '''Opens a file specified by the filename, including the directory and creates dataframe for the respective file. Subsequently, 
    this function analyses the chapter with regard to our criteria: 1) Length of the text 2) direct bible quote 3) percentage of text that is a quote'''
    #Öffnen der Datei "filename"
    file_title = filename.split(".")[0]
    with open(filename, 'r') as f:
        file = f.read()   
    soup = bs.BeautifulSoup(file, 'lxml')
    #Wählen aller ref-Elemente, die das Attribut cRef haben
    Refs = soup.select('ref[cRef]')
    # Konvertierung der Daten in data. Dies ist die Grundlage für die Auswertungen
    data = []
    for ref in Refs:
        if ref.get('cref').split('_')[0] in AT:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'at', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'at', ref.getText(), 'not a direct quote'))
        else:
            if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling('q'):
                data.append((ref.get('cref') , 'nt', ref.getText() , ref.parent.find_previous_sibling('q').get_text(' ',strip=True)))
            else:
                data.append((ref.get('cref'), 'nt', ref.getText(), 'not a direct quote'))
    # der eigentliche Dataframe wird gesetzt
    df_data = pd.DataFrame(data)
    # der Header wird gesetzt.
    df_data.columns = ['kuerzel', 'atnt', 'stelle', 'zitat']



    # Hier wird eine Liste der wörtlichen Zitate erstellt
    qs = []
    for ref in Refs:
        if ref.parent.find_previous_sibling() == ref.parent.find_previous_sibling("q"):
            qs.append(ref.parent.find_previous_sibling("q").get_text(" ", strip=True))



    # Hier eine Liste der zitierten Stelle 
    stellen = []
    for i in range (0, len(df_data)):
        if df_data['zitat'][i] != 'not a direct quote':
            stellen.append(df_data['stelle'][i])
    #print(stellen)
    
    # Zählen der Gesamtlänge des zitierten Textes; ohne Leerzeilen und Zeilenumbrüche 
    quote_len = 0
    for i in range(len(qs)):
        quote_len +=len(qs[i].replace(" ","").replace("\n", ""))

    #ges = bs.BeautifulSoup(file, 'lxml')
    ges_soup = soup.select('div')
    #print(len(ges_soup[2].text))
    notes = soup.body.select("note")
    supplied = soup.body.select("supplied")
    
    
    # Zählen der Gesamtlänge des Textes; ohne Leerzeilen und Zeilenumbrüche
    total_len=0
    for i in range(len(ges_soup)):
        total_len += len(ges_soup[i].getText().replace(" ", "").replace("\n", ""))
     
   # Berechnung der Länge des Textes aller note-Elemente 
    no_len = 0
    for i in range(len(notes)):
        #print(len(notes[i].getText()))
        no_len += len(notes[i].getText().replace(" ","").replace("\n", ""))
    #print(no_len)
    supplied_len = 0
    for i in range(len(supplied)):
        supplied_len += len(supplied[i].getText().replace(" ", "").replace("\n", ""))


    print(f"---{filename}---")
    print(f"Länge inkl. <note>: {total_len}")
    print(f"supplied: {supplied_len}")
    print(f"Länge d. <note>-Inhalte: {no_len}")
    print("Gesamtlänge:" +str(total_len-no_len-supplied_len))
    print(f"Anzahl an erfassten Bibelstellen: {len(Refs)}")
    print(f"wörtliche Zitate: {len(stellen)}")
    #p.pprint(f"Liste d. wörtlichen Zitate:{stellen}")
    print("Zitatlänge :"+ str(quote_len))    
    print("Prozentualer Anteil: " +(str(percent_calc(quote_len, total_len))))
    lines = [f"Anzahl an erfassten Bibelstellen: {len(Refs)}", f"wörtliche Zitate: {len(stellen)}", f"Liste d. wörtl.: {stellen}",f"Länge inkl. note: {total_len}" ,f"Gesamtlänge: {str(total_len-no_len-supplied_len)}", f"Länge d. erf. <note>-Inhalte: {no_len}", f"Zitatlänge :{str(quote_len)}",f"Prozentualer Anteil: {(str(percent_calc(quote_len, total_len)))}"]
    # Speichern der obigen Ausgabe harter Fakten in einer Text-Datei mit dem Titel der ausgewählten Datei. 
    with open(f'Visualisierungen/{file_title}', 'w') as f2:
        for line in lines:
            f2.write(line)
            f2.write("\n")
    # Ausgabe, Speichern und Schließen der einzelen Diagramme. 
    print(df_data["kuerzel"].value_counts().plot(figsize=(8,8), kind="pie", legend=False, title=f"{filename}"))
    pplt.savefig(f'Visualisierungen/{file_title}.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["atnt"].value_counts().plot(figsize=(8,8), kind="bar", legend=False, title=f"{filename}", color=["blue", "red"]))
    pplt.savefig(f'Visualisierungen/{file_title}_atnt.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["kuerzel"].value_counts()[:5].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename}  @cRef)", color="purple"))
    pplt.savefig(f'Visualisierungen/{file_title}_cref.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    print(df_data["stelle"].value_counts()[:10].plot(figsize=(8,8), kind="barh", legend=False, title=f"{filename} nach Stellen", color="red"))
    pplt.savefig(f'Visualisierungen/{file_title}_nach_stellen.jpg')
    pplt.clf()
    pplt.cla()
    pplt.close()
    
    

In [3]:
def no_bible_quote(filename):
    '''Opens a file specified by the filename, including the directory and creates dataframe for the respective file. Subsequently, this function analyses the chapter with regard to our criteria'''
    file_title = filename.split(".")[0]
    with open(filename, 'r') as f:
        file = f.read() 
    ges = bs.BeautifulSoup(file, 'lxml')
    ges_soup = ges.select('div')
    notes = ges.find_all("note")
    supplied = soup.body.select("supplied")
    # Zählen der Gesamtlänge des Textes; ohne Leerzeilen und Zeilenumbrüche
    total_len=0
    for i in range(len(ges_soup)):
        total_len += len(ges_soup[i].getText().replace(" ", "").replace("n", ""))
     
   # Berechnung der Länge des Textes aller note-Elemente 
    no_len = 0
    for i in range(len(notes)):
        #print(len(notes[i].getText()))
        no_len += len(notes[i].getText())
    print(no_len)
    
    supplied_len = 0
    for i in range(len(supplied)):
        supplied_len += len(supplied[i].getText().replace(" ", "").replace("\n", ""))

    print("Gesamtlänge:" +str(total_len-no_len-supplied_len))

# Daten für das gesamte Interim

In [4]:

analyse('interim4.xml')


---interim4.xml---
Länge inkl. <note>: 123612
supplied: 43
Länge d. <note>-Inhalte: 24805
Gesamtlänge:98764
Anzahl an erfassten Bibelstellen: 283
wörtliche Zitate: 114
Zitatlänge :9720
Prozentualer Anteil: 7.863314241335792
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


In [5]:
analyse('chapters/05.xml')

/home/kevin/PycharmProjects/Graz_Bibelstellen/venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


---chapters/05.xml---
Länge inkl. <note>: 826
supplied: 1
Länge d. <note>-Inhalte: 234
Gesamtlänge:591
Anzahl an erfassten Bibelstellen: 3
wörtliche Zitate: 0
Zitatlänge :0
Prozentualer Anteil: 0.0
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


In [6]:
with open('interim.xml', 'r') as f:
    file = f.read()   
ges2 = bs.BeautifulSoup(file, 'lxml')
ges_soup2 = ges2.select('div')
print(len(ges_soup2[0]))
print(ges_soup2[0].getText(" ", strip=True))

29
Der Römischen Keyserlichen Majestat Erklärung, wie es der Reli - gion halben imm heyligen Reich biß zu Außtrag deß gemeinen Concilii gehalten werden sol, auff dem Reichß - - tag zu Augspurg den XV. Maii im M. D. XLVIII. Jar publiciert unnd eröffnet unnd von ge - - meinen Stenden angenommen. Christo Auspice Plus Ultra. Cum Gratia et Privilegio etc.


In [11]:
for i in range(1,27):
    if i <10:
        print(f"analyse('chapters/0{i}.xml')")
    else:
        print(f"analyse('chapters/{i}.xml')")
    i +=1

analyse('chapters/01.xml')
analyse('chapters/02.xml')
analyse('chapters/03.xml')
analyse('chapters/04.xml')
analyse('chapters/05.xml')
analyse('chapters/06.xml')
analyse('chapters/07.xml')
analyse('chapters/08.xml')
analyse('chapters/09.xml')
analyse('chapters/10.xml')
analyse('chapters/11.xml')
analyse('chapters/12.xml')
analyse('chapters/13.xml')
analyse('chapters/14.xml')
analyse('chapters/15.xml')
analyse('chapters/16.xml')
analyse('chapters/17.xml')
analyse('chapters/18.xml')
analyse('chapters/19.xml')
analyse('chapters/20.xml')
analyse('chapters/21.xml')
analyse('chapters/22.xml')
analyse('chapters/23.xml')
analyse('chapters/24.xml')
analyse('chapters/25.xml')
analyse('chapters/26.xml')


In [12]:
analyse('chapters/01.xml')
analyse('chapters/02.xml')
analyse('chapters/03.xml')
analyse('chapters/04.xml')
analyse('chapters/05.xml')
analyse('chapters/06.xml')
analyse('chapters/07.xml')
analyse('chapters/08.xml')
analyse('chapters/09.xml')
analyse('chapters/10.xml')
analyse('chapters/11.xml')
analyse('chapters/12.xml')
analyse('chapters/13.xml')
#analyse('chapters/14.xml')
analyse('chapters/15.xml')
analyse('chapters/16.xml')
analyse('chapters/17.xml')
analyse('chapters/18.xml')
analyse('chapters/19.xml')
analyse('chapters/20.xml')
analyse('chapters/21.xml')
analyse('chapters/22.xml')
analyse('chapters/23.xml')
analyse('chapters/24.xml')
#analyse('chapters/25.xml')
analyse('chapters/26.xml')

/home/kevin/PycharmProjects/Graz_Bibelstellen/venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


---chapters/01.xml---
Länge inkl. <note>: 1372
supplied: 1
Länge d. <note>-Inhalte: 284
Gesamtlänge:1087
Anzahl an erfassten Bibelstellen: 2
wörtliche Zitate: 1
Zitatlänge :45
Prozentualer Anteil: 3.279883381924198
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
---chapters/02.xml---
Länge inkl. <note>: 2962
supplied: 1
Länge d. <note>-Inhalte: 497
Gesamtlänge:2464
Anzahl an erfassten Bibelstellen: 14
wörtliche Zitate: 1
Zitatlänge :75
Prozentualer Anteil: 2.5320729237002024
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


/home/kevin/PycharmProjects/Graz_Bibelstellen/venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
---chapters/03.xml---
Länge inkl. <note>: 2163
supplied: 1
Länge d. <note>-Inhalte: 339
Gesamtlänge:1823
Anzahl an erfassten Bibelstellen: 8
wörtliche Zitate: 1
Zitatlänge :173
Prozentualer Anteil: 7.998150716597319
AxesSubplot(0.1275,0.11;0.77x0.77)


/home/kevin/PycharmProjects/Graz_Bibelstellen/venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


...

---chapters/23.xml---
Länge inkl. <note>: 5835
supplied: 2
Länge d. <note>-Inhalte: 1023
Gesamtlänge:4810
Anzahl an erfassten Bibelstellen: 19
wörtliche Zitate: 8
Zitatlänge :831
Prozentualer Anteil: 14.241645244215938
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
---chapters/24.xml---
Länge inkl. <note>: 7486
supplied: 2
Länge d. <note>-Inhalte: 2475
Gesamtlänge:5009
Anzahl an erfassten Bibelstellen: 3
wörtliche Zitate: 0
Zitatlänge :0
Prozentualer Anteil: 0.0
AxesSubplot(0.1275,0.11;0.77x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)


/home/kevin/PycharmProjects/Graz_Bibelstellen/venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
---chapters/26.xml---
Länge inkl. <note>: 9420
supplied: 2
Länge d. <note>-Inhalte: 1230
Gesamtlänge:8188
Anzahl an erfassten Bibelstellen: 11
wörtliche Zitate: 2
Zitatlänge :73
Prozentualer Anteil: 0.7749469214437368
AxesSubplot(0.1275,0.11;0.77x0.77)


/home/kevin/PycharmProjects/Graz_Bibelstellen/venv/lib/python3.10/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
AxesSubplot(0.125,0.11;0.775x0.77)
